<a href="https://colab.research.google.com/github/DanZter/BERT_TRAINING_sentiment_model/blob/master/BERT__sentiment_model_with_flaskAPI_(TPU).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
VERSION = "1.5"  #@param ["1.5" , "20200325", "nightly"]
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py --version $VERSION

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4994  100  4994    0     0  71342      0 --:--:-- --:--:-- --:--:-- 71342
Updating... This may take around 2 minutes.
Updating TPU runtime to pytorch-1.5 ...
     |████████████████████████████████| 61kB 3.1MB/s 
Uninstalling torch-1.5.1+cu101:
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12
Done updating TPU runtime
  Successfully uninstalled torch-1.5.1+cu101
Uninstalling torchvision-0.6.1+cu101:
  Successfully uninstalled torchvision-0.6.1+cu101
Copying gs://tpu-pytorch/wheels/torch-1.5-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 79.0 MiB/ 79.0 MiB]                                                
Operation completed over 1 objects/79.0 MiB.                                     
Copying gs://tpu-pyto

In [3]:
# imports pytorch
import torch

# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.2.0
Running on TPU  ['10.16.170.202:8470']
INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [5]:
!pip install torch
!pip install transformers
!pip install pandas
!pip install -U scikit-learn

     |████████████████████████████████| 778kB 3.2MB/s 
     |████████████████████████████████| 890kB 16.9MB/s 
     |████████████████████████████████| 3.0MB 18.9MB/s 
     |████████████████████████████████| 1.1MB 46.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=c092bdc943f721bcee35711859c647ab2431386dd39b3d442445667ef9b2ecac
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 6.9MB 3.2MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [6]:
import pandas as pd
import numpy as np
import transformers
from transformers import AdamW, get_linear_schedule_with_warmup
import torch.nn as nn
from sklearn import model_selection
from scipy import stats
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
# import logging
# logging.basicConfig(level=logging.ERROR)

In [7]:
MAX_LEN = 256
# TRAIN_BATCH_SIZE = 8
# VALID_BATCH_SIZE = 4
# EPOCHS = 10
# ACCUMULATION = 2
BERT_PATH = "/content/drive/My Drive/Colab Notebooks/input/bert_base_uncased"
TRAINING_FILE = "/content/drive/My Drive/Colab Notebooks/input/IMDB Dataset.csv"
TOKENIZER = transformers.BertTokenizer.from_pretrained(BERT_PATH, do_lower_case=True)
MODEL_PATH ="/content/drive/My Drive/Colab Notebooks/models/bert_sentiment_model.bin"


In [8]:
class BERTDataset:
    def __init__(self, review, target):
        self.review = review                     # the review text, a list
        self.target = target                     # 0 or 1, a list
        self.tokenizer = TOKENIZER
        self.max_len = MAX_LEN

    def __len__(self):                           # returns the total length of data set
        return len(self.review)

    def __getitem__(self, item):                 # takes an 'item' and returns tokenizer of that item from data set
        review = str(self.review[item])          # converts everything to str incase there exists numbers etc.
        review = " ".join(review.split())        # removes all unnecessary space

        inputs = self.tokenizer.encode_plus(     # encode_plus can encode 2 strings at a time
            review,
            None,                                # since we use only 1 string at a time
            add_special_tokens=True,             # adds cld, sep tokens
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True
        )

        ids = inputs["input_ids"]
        mask = inputs["attention_mask"]
        token_type_ids = inputs["token_type_ids"] # since only 1 string token_type_ids are same and unnecessary

        padding_length = self.max_len - len(ids)  # for bert we pad on the right side
        ids = ids + ([0] * padding_length)        # zero times the padding length
        mask = mask + ([0] * padding_length)
        token_type_ids = token_type_ids + ([0] * padding_length)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'target': torch.tensor(self.target[item], dtype=torch.float)
        }
    """ if we have 2 target outputs then set to torch.long,
    depends on loss function also, from cross-entropy we should use torch.long"""

In [9]:
def loss_fn(outputs, target):
    return nn.BCEWithLogitsLoss()(outputs, target.view(-1, 1))

def train_fn(data_loader, model, optimizer, device, scheduler=None):
    model.train()

    for bi, d in enumerate(data_loader):
        ids = d["ids"]
        mask = d["mask"]
        token_type_ids = d["token_type_ids"]
        target = d["target"]

        ids = ids.to(device, dtype=torch.long)              # send to TPU device
        mask = mask.to(device, dtype=torch.long)
        token_type_ids = token_type_ids.to(device, dtype=torch.long)
        target = target.to(device, dtype=torch.float)

        optimizer.zero_grad()
        outputs = model(
            ids=ids,
            mask=mask,
            token_type_ids=token_type_ids
        )

        loss = loss_fn(outputs, target)        # find loss
        loss.backward()                         # backward propagation

        xm.optimizer_step(optimizer)
        if scheduler is not None:
            scheduler.step()
        if bi % 10 == 0:
            xm.master_print(f"batch_index={bi}, loss={loss}")

        """ stop the optimizer only after a certain number of accumulation steps """


def eval_fn(data_loader, model, device):
    model.eval()
    fin_target = []                         # final targets
    fin_outputs = []                        # final outputs
    with torch.no_grad():
        for bi, d in enumerate(data_loader):
            ids = d["ids"]
            mask = d["mask"]
            token_type_ids = d["token_type_ids"]
            target = d["target"]

            ids = ids.to(device, dtype=torch.long)              # send to cuda device
            mask = mask.to(device, dtype=torch.long)
            token_type_ids = token_type_ids.to(device, dtype=torch.long)
            target = target.to(device, dtype=torch.float)

            outputs = model(
                ids=ids,
                mask=mask,
                token_type_ids=token_type_ids
            )
            # loss = loss_fn(outputs, targets)        # find loss, its bettwer to evaluate loss in eval fn

            fin_target.extend(target.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

    return fin_outputs, fin_target

In [10]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(BERT_PATH)
        self.bert_drop = nn.Dropout(0.3)
        self.out = nn.Linear(768, 1)
        """ 768: bert we use have 768 features | 1: binary classification
        if we use 2, we need to change the loss function"""

    def forward(self, ids, mask, token_type_ids):
        _, o2 = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids
        )
        """ We have 2 outputs from a BERT model
         o1(last hidden state): is the sequence of hidden states. eg. if we have 512 tokens (MAX_LEN), 
         we have 512 vectors of size 768 for each batch. We can use out1 to max pooling or averge pooling
         o2(pooler output from bert pooler layer): we get vector of size 768 for each sample in batch"""
        bo = self.bert_drop(o2)                                 # drop-out
        output = self.out(bo)                                   # linear-layer
        return output

In [11]:
from sklearn import metrics

def run(index, flags):

    flags['MAX_LEN'] = 256
    flags['TRAIN_BATCH_SIZE'] = 32  # 9.47min for BS=8
                                    # 5.54min for BS=32
    flags['VALID_BATCH_SIZE'] = 16
    flags['EPOCHS'] = 10
    flags['seed'] = 1234  
    torch.manual_seed(flags['seed'])

    dfx = pd.read_csv(TRAINING_FILE).fillna("none")
    dfx.sentiment = dfx.sentiment.apply(  # can use label encoding
        lambda x: 1 if x == "positive" else 0  # can use map fn
    )

    df_train, df_valid = model_selection.train_test_split(
        dfx,
        test_size=0.1,
        random_state=42,
        stratify=dfx.sentiment.values  # when split both train and val have same positive to negative sample ratio
    )

    df_train = df_train.reset_index(drop=True)  # 0 to length of df_train
    df_valid = df_valid.reset_index(drop=True)  # 0 to length of df_valid

    train_dataset = BERTDataset(
        review=df_train.review.values,
        target=df_train.sentiment.values
    )
    train_sampler = torch.utils.data.DistributedSampler(
       train_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal(),
       shuffle=True 
    )

    train_data_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=flags['TRAIN_BATCH_SIZE'],
        sampler=train_sampler
    )
####################################################
    valid_dataset = BERTDataset(
        review=df_valid.review.values,
        target=df_valid.sentiment.values
    )
    valid_sampler = torch.utils.data.DistributedSampler(
       valid_dataset,
       num_replicas=xm.xrt_world_size(),
       rank=xm.get_ordinal()
    )
    valid_data_loader = torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=flags['VALID_BATCH_SIZE'],
        sampler=valid_sampler
    )

    # device = "cuda"                   # cuda
    device = xm.xla_device()            # tpu

    model = BERTBaseUncased().to(device)
    # lr = 3e-5 *xm.xrt_world_size()
    param_optimizer = list(model.named_parameters())  # specify parameters to train
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.001},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    """ These parameters are adjustable, we should take a look at different layers and
    the decay we want, how much learning rate etc."""

    num_train_steps = int(len(df_train) / flags['TRAIN_BATCH_SIZE'] / xm.xrt_world_size() * flags['EPOCHS'])
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )

    # model = nn.DataParallel(model)              # converting to multi gpu model

    best_accuracy = 0
    for epoch in tqdm(range(flags['EPOCHS']), total=flags['EPOCHS']):

        para_loader = pl.ParallelLoader(train_data_loader, [device])
        train_fn(para_loader.per_device_loader(device), model, optimizer, device, scheduler)

        para_loader = pl.ParallelLoader(valid_data_loader, [device])
        outputs, target = eval_fn(para_loader.per_device_loader(device), model, device)
        outputs = np.array(outputs) >= 0.5
        accuracy = metrics.accuracy_score(target, outputs)
        xm.master_print(f"Accuracy score = {accuracy}")
        xm.save(model.state_dict(), MODEL_PATH)          # tpu  # saving the model only if it improves


# flags = {}
# xmp.spawn(run, args=(flags,), nprocs=8, start_method='fork')

In [12]:
!pip install -U Flask
!pip install flask-ngrok

Requirement already up-to-date: Flask in /usr/local/lib/python3.6/dist-packages (1.1.2)


In [ ]:
import flask
from flask import Flask
from flask import request, render_template

import torch
import flask
import time
from flask import Flask
from flask import request

import functools
import torch.nn as nn

from flask_ngrok import run_with_ngrok

app = Flask(__name__)     # initialize flask app
run_with_ngrok(app)
MODEL = None
DEVICE = xm.xla_device()
PREDICTION_DICT = dict()
# memory = joblib.Memory("../input/", verbose=0)

@app.route("/")
def home():
  return render_template("index.html")

def predict_from_cache(sentence):
    if sentence in PREDICTION_DICT:
        return PREDICTION_DICT[sentence]
    else:
        result = sentence_prediction(sentence)
        PREDICTION_DICT[sentence] = result
        return result


@memory.cache
def sentence_prediction(sentence):
    tokenizer = TOKENIZER
    max_len = MAX_LEN
    review = str(sentence)
    review = " ".join(review.split())     # removes all unnecessary space

    inputs = tokenizer.encode_plus(       # encode_plus can encode 2 strings at a time
        review,
        None,                             # since we use only 1 string at a time
        add_special_tokens=True,          # adds cls, sep tokens
        max_length=max_len,
        truncation=True
    )

    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]      # since only 1 string token_type_ids are same and unnecessary

    padding_length = max_len - len(ids)            # for bert we pad on the right side
    ids = ids + ([0] * padding_length)             # zero times the padding length
    mask = mask + ([0] * padding_length)
    token_type_ids = token_type_ids + ([0] * padding_length)

    ids = torch.tensor(ids, dtype=torch.long).unsqueeze(0)    # since dataloader always returns batches
    mask = torch.tensor(mask, dtype=torch.long).unsqueeze(0)
    token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0)

    ids = ids.to(DEVICE, dtype=torch.long)          # send to tpu DEVICE
    token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
    mask = mask.to(DEVICE, dtype=torch.long)

    outputs = MODEL(ids=ids, mask=mask, token_type_ids=token_type_ids)

    outputs = torch.sigmoid(outputs).cpu().detach().numpy()
    return outputs[0][0]          # since outputs will be 2-Dimensional but there is only 1 value


@app.route("/predict", methods=['POST'])            # creating end point using flask
def predict():                                      # predict function
    sentence = [str(x) for x in request.form.values()]
    sentence = sentence[0]
    # sentence = str(request.form.values())
    start_time = time.time()
    positive_prediction = sentence_prediction(sentence)
    negative_prediction = 1 - positive_prediction
    response = {}
    response["response"] = {
        "positive": str(positive_prediction),
        "negative": str(negative_prediction),
        "sentence": str(sentence),
        "time_taken": str(time.time() - start_time),
    }
    return render_template("index.html", sentence = r"Your Sentence = '{}'".format(sentence),
    prediction_text="Sentiment of the sentence is :", positive=  "{}% POSITIVE".format(round(positive_prediction*100,2)),
    negative = "{}% NEGATIVE".format(round(negative_prediction*100,2)))




if __name__ == "__main__":
    MODEL = BERTBaseUncased()
    MODEL.load_state_dict(torch.load(MODEL_PATH))
    MODEL.to(DEVICE)
    MODEL.eval()
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5a0ea38a2576.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/Jul/2020 01:39:13] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2020 01:39:13] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2020 01:39:14] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Jul/2020 01:39:16] "POST /predict HTTP/1.1" 200 -


In [ ]:
# app = Flask(__name__)     # initialize flask app
# run_with_ngrok(app)       # ngrok for colab VM

# MODEL = None
# DEVICE = xm.xla_device()
# PREDICTION_DICT = dict()
# memory = joblib.Memory("../input/", verbose=0)


# def predict_from_cache(sentence):
#     if sentence in PREDICTION_DICT:
#         return PREDICTION_DICT[sentence]
#     else:
#         result = sentence_prediction(sentence)
#         PREDICTION_DICT[sentence] = result
#         return result


# @memory.cache
# def sentence_prediction(sentence):
#     tokenizer = TOKENIZER
#     max_len = MAX_LEN
#     review = str(sentence)
#     review = " ".join(review.split())     # removes all unnecessary space

#     inputs = tokenizer.encode_plus(       # encode_plus can encode 2 strings at a time
#         review, 
#         None,                             # since we use only 1 string at a time
#         add_special_tokens=True,          # adds cls, sep tokens
#         max_length=max_len
#     )

#     ids = inputs["input_ids"]
#     mask = inputs["attention_mask"]
#     token_type_ids = inputs["token_type_ids"]      # since only 1 string token_type_ids are same and unnecessary

#     padding_length = max_len - len(ids)            # for bert we pad on the right side
#     ids = ids + ([0] * padding_length)             # zero times the padding length
#     mask = mask + ([0] * padding_length)
#     token_type_ids = token_type_ids + ([0] * padding_length)

#     ids = torch.tensor(ids, dtype=torch.long).unsqueeze(0)    # since dataloader always returns batches
#     mask = torch.tensor(mask, dtype=torch.long).unsqueeze(0)
#     token_type_ids = torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0)

#     ids = ids.to(DEVICE, dtype=torch.long)          # send to tpu DEVICE
#     token_type_ids = token_type_ids.to(DEVICE, dtype=torch.long)
#     mask = mask.to(DEVICE, dtype=torch.long)

#     outputs = MODEL(ids=ids, mask=mask, token_type_ids=token_type_ids)

#     outputs = torch.sigmoid(outputs).cpu().detach().numpy()
#     return outputs[0][0]          # since outputs will be 2-Dimensional but there is only 1 value


# @app.route("/predict")            # creating end point using flask
# def predict():                    # predict function
#     sentence = request.args.get("sentence")
#     start_time = time.time()
#     positive_prediction = sentence_prediction(sentence)
#     negative_prediction = 1 - positive_prediction
#     response = {}
#     response["response"] = {
#         "positive": str(positive_prediction),
#         "negative": str(negative_prediction),
#         "sentence": str(sentence),
#         "time_taken": str(time.time() - start_time),
#     }
#     return flask.jsonify(response)
    


# if __name__ == "__main__":
#     MODEL = BERTBaseUncased()
#     # MODEL = nn.DataParallel(MODEL)      # for multicore parallel training
#     MODEL.load_state_dict(torch.load(MODEL_PATH))
#     MODEL.to(DEVICE)
#     MODEL.eval()
#     app.run()